<img src="https://raw.githubusercontent.com/maverick-faus/Files/master/DeepDeamon2.png" alt="drawing" width="100px"/>
# Red BNB - Testing
TT: Deep Deep Dishes  
Food recommendation System  
ESCOM  
By Faus - 2K18


**Instalación de biblioteca para acceso a Google Drive**

In [0]:
import tensorflow as tf
import numpy as np
import pickle 
from sklearn.metrics import confusion_matrix

In [6]:
!pip install PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


    100% |████████████████████████████████| 993kB 7.4MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileIdLabelmap = '1KgchliGJ9QFqoqLmsWs0dwoRPuOtrV5v'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("BNB1.ckpt.data-00000-of-00001")

fileIdLabelmap = '1txXYJ41n5Z_opzuF5Gw6G2R064856lXB'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("BNB1.ckpt.index")

fileIdLabelmap = '1i6WuNDK4md4LdlaMgqMu-EwuL3Ea6elX'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("BNB1.ckpt.meta")

fileIdLabelmap = '1bhmQcD7lOYo3qld_0-2cOliC23P4mYR0'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("BNB_test.pickle")

In [14]:
!ls

adc.json		       BNB1.ckpt.index	BNB_test.pickle  sample_data
BNB1.ckpt.data-00000-of-00001  BNB1.ckpt.meta	drive


In [0]:
def deep_neural_convolutional_class(
    batch_size=20,
    image_size=[200,200],
    Drop_prob=1.0,
    learning_rate = 1e-3,
    n_nodes_hl0 = 2000,
    n_nodes_hl1 = 1000,
    n_nodes_hl2 = 500,
    n_nodes_hl3 = 100,
    n_classes=2
    ):
    
    reset_graph()
    #Place holder de entrada 
    x= tf.placeholder(tf.float32,[batch_size,image_size[0],image_size[1],3], name='placeholder_img_entrada')
    y=tf.placeholder(tf.float32,name='placeholder_one_hot')
  
    #Diccionario de pesos convolucionales 
    with tf.name_scope('pesos_bias') as scope1:
        weigths={"w_conv1":tf.Variable(tf.random_normal([5,5,3,32]),name='Pesos_1_32'),
                 "w_conv2":tf.Variable(tf.random_normal([5,5,32,64]),name='Pesos_1_64'),
                 "w_conv3":tf.Variable(tf.random_normal([3,3,64,128]),name='Pesos_1_128'),   
                 "w_conv4":tf.Variable(tf.random_normal([5,5,128,256]),name='Pesos_1_256'),
                 #"w_conv5":tf.Variable(tf.random_normal([5,5,256,512]),name='Pesos_1_512'),
                 #"w_conv6":tf.Variable(tf.random_normal([5,5,512,1024]),name='Pesos_1_1024'),
                }
        #Diccionario de bias
        biases={"b_conv1":tf.Variable(tf.random_normal([32]),name='Bias_1_32'),
                "b_conv2":tf.Variable(tf.random_normal([64]),name='Bias_1_64'),
                "b_conv3":tf.Variable(tf.random_normal([128]),name='Bias_1_128'),
                "b_conv4":tf.Variable(tf.random_normal([256]),name='Bias_1_256'),
                #"b_conv5":tf.Variable(tf.random_normal([512]),name='Bias_1_512'),
                #"b_conv6":tf.Variable(tf.random_normal([1024]),name='Bias_1_1024'),
               }

    #Extractor de características
    with tf.name_scope('capas_conv') as scope2:
        conv1=tf.nn.relu(conv2d(x,weigths["w_conv1"],'Capa_Conv_1','SAME')+biases["b_conv1"],name='Func_relu_1')
        conv1=tf.nn.dropout(conv1,Drop_prob)
        conv1=maxpool2d(conv1,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 100x100x32
        print(conv1)

        conv2=tf.nn.relu(conv2d(conv1,weigths["w_conv2"],'Capa_Conv_2','SAME')+biases["b_conv2"],name='Func_relu_2')
        conv2=tf.nn.dropout(conv2,Drop_prob)
        conv2=maxpool2d(conv2,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 50x50x64
        print(conv2)

        conv3=tf.nn.relu(conv2d(conv2,weigths["w_conv3"],'Capa_Conv_3','VALID')+biases["b_conv3"],name='Func_relu_3')
        conv3=tf.nn.dropout(conv3,Drop_prob)
        conv3=maxpool2d(conv3,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 24x24x128
        print(conv3)

        conv4=tf.nn.relu(conv2d(conv3,weigths["w_conv4"],'Capa_Conv_4','SAME')+biases["b_conv4"],name='Func_relu_4')
        conv4=tf.nn.dropout(conv4,Drop_prob)
        conv4=maxpool2d(conv4,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 12x12x256
        print(conv4)

        #conv5=tf.nn.relu(conv2d(conv4, weigths["w_conv5"],'Capa_Conv_5','SAME')+biases["b_conv5"],name='Func_relu_5')
        #conv5=tf.nn.dropout(conv5,Drop_prob)
        #conv5=maxpool2d(conv5,ks=[1,2,2,1],st=[1,2,2,1])
        #vector para clasificar de 6x6x512
        #print(conv5)
        
        #conv6=tf.nn.relu(conv2d(conv5, weigths["w_conv6"],'Capa_Conv_6','SAME')+biases["b_conv6"],name='Func_relu_6')
        #conv6=tf.nn.dropout(conv6,Drop_prob)
        #conv6=maxpool2d(conv6,ks=[1,4,4,1],st=[1,4,4,1])
        #vector para clasificar de 1x1x1024
        #print(conv6)

        #Embeding, son las caracteristicas fonales que se pasarán al MLP o red completamente conectada para clasifiacar
        embdeding=tf.reshape(conv4,[batch_size,12*12*256],name='Embeding')
        print(embdeding)
    
    #Red perceptron, declaración de capas, son diccionarios de pesos y bias.
    with tf.name_scope('capas_clasificador') as scope3:
        hidden_0_layer = {'weights':tf.Variable(tf.random_normal([12*12*256, n_nodes_hl0]),name='Capa_oculta_pesos_0'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl0]),name='Capa_oculta_bias_0')}

        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl0, n_nodes_hl1]),'Capa_oculta_pesos_1'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='Capa_oculta_bias_1')}

        hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]),'Capa_oculta_pesos_2'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]),name='Capa_oculta_bias_2')}
        
        hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),'Capa_oculta_pesos_3'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]),name='Capa_oculta_bias_3')}

        output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),'Capa_salida_pesos'),
                        'biases':tf.Variable(tf.random_normal([n_classes]),name='Capa_salida_bias'),}
    
    #W*P + B 
    with tf.name_scope('op_clasificador') as scope4:
        
        l0 = tf.add(tf.matmul(embdeding,hidden_0_layer['weights'],name='Matmul_l0'), hidden_0_layer['biases'],name='Suma_Pesos_Bias_0')
        l0 = tf.nn.relu(l0,name='l0_relu_0')

        l1 = tf.add(tf.matmul(l0,hidden_1_layer['weights'],name='Matmul_l1'), hidden_1_layer['biases'],name='Suma_Pesos_Bias_1')
        l1 = tf.nn.relu(l1,name='l1_relu_1')

        l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights'],name='Matmul_l2'), hidden_2_layer['biases'],name='Suma_Pesos_Bias_2')
        l2 = tf.nn.relu(l2,name='l2_relu_2')
        
        l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights'],name='Matmul_l3'), hidden_3_layer['biases'],name='Suma_Pesos_Bias_3')
        l3 = tf.nn.relu(l3,name='l3_relu_3')

        output = tf.matmul(l3,output_layer['weights'],name='Matmul_out') + output_layer['biases']
    
    # Declarando la funcion de costo y entrenamiento
    #Reduce mean, reduce la dimension del tensor en un promedio es decir hace el promedio del costo o error
    
    
    with tf.name_scope('costo_y_optimizador') as scope5:
        cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y,name='softmax_cross_entropy_with_logits1'),name='reduce_mean1')
        optimizer = tf.train.AdamOptimizer(name='Adam',learning_rate=learning_rate).minimize(cost,name='minimo')
    
    
    tf.summary.scalar("costo",cost)
    correct = tf.equal(tf.argmax(output,1),tf.argmax(y,1),name='valores_correctos')
    accuracy = tf.reduce_mean(tf.cast(correct,'float'),name='Promedio_exactitud') #porcentaje de error
    
    tf.summary.scalar("accuracy",accuracy)
    summaries = tf.summary.merge_all()
    
    return dict(
        x = x,
        y=y,
        embeding=conv4,
        output=output,
        saver = tf.train.Saver(),
        total_loss = cost,
        train_step = optimizer,
        summaries = summaries,
        accuracy = accuracy
    )

In [0]:
def conv2d(x, W,name,padd,strid=[1,1,1,1]):
    #El stride de esa función no reduce el tamaño de la imagen
    return tf.nn.conv2d(x, W, strides=strid, padding=padd,name=name)

def maxpool2d(x,ks,st):
    #           El st de esta función reduce la imagen a la mitad
    return tf.nn.max_pool(x, ksize=ks, strides=st, padding='SAME')

In [0]:
def reset_graph():
    #Limpiamos la gráfic
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:
def check_net(g, test, checkpoint):
   with tf.Session() as sess:
        writer = tf.summary.FileWriter("./CNNClass")
        tf.summary.FileWriter.add_graph(writer,sess.graph)
        #sess.run(tf.initialize_all_variables())
        g['saver'].restore(sess, checkpoint)
        
        feed_dict={g['x']: test[:,0].tolist(),g['y']: test[:,1].tolist()}
        preds,accuracy = sess.run([g['output'],g['accuracy']], feed_dict)
        return preds,accuracy


In [28]:
path= 'BNB_test'
data1=np.array(pickle.load(open(path+".pickle","rb"))) #np.load()
print(data1.shape)
g=deep_neural_convolutional_class(batch_size = 200)#Batch de prueba
pred,accuracy=check_net(g,data1,"./BNB1.ckpt")

(200, 2)
Tensor("capas_conv/MaxPool:0", shape=(200, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(200, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(200, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(200, 12, 12, 256), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(200, 36864), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from ./BNB1.ckpt


(200,)


In [29]:
print(accuracy)

0.665


In [30]:
pred2 = pred
print(pred2)

[[ 8.47849718e+11  1.84942395e+12]
 [-1.05680218e+13 -2.78742512e+12]
 [-2.05933616e+12 -1.37287172e+12]
 [ 3.55721360e+12 -1.04285254e+12]
 [-6.03008375e+12  1.99326328e+12]
 [-1.47541144e+12  2.51487584e+11]
 [-2.83930264e+12 -1.87330488e+12]
 [-5.43300780e+12 -1.64792081e+12]
 [-1.28471532e+10 -1.12147982e+12]
 [ 2.74588343e+12  2.25798757e+12]
 [-4.63904493e+12 -2.53076754e+12]
 [-8.92342056e+12 -8.06786643e+12]
 [-3.20123961e+12  6.37899217e+12]
 [-7.53505298e+12 -4.21258068e+12]
 [ 8.05872075e+11 -2.17332423e+12]
 [-2.74769799e+12 -2.68587919e+12]
 [-7.93193690e+12 -3.89496884e+12]
 [ 6.07209705e+12  2.40205798e+12]
 [-2.30419386e+12 -4.91070371e+12]
 [-3.28244763e+12  3.00987528e+12]
 [ 3.96188031e+12 -1.35694018e+12]
 [-6.33085100e+11 -7.46367903e+12]
 [ 1.59072125e+11 -4.96027697e+11]
 [-5.19563641e+12 -4.27076144e+12]
 [ 2.71401563e+12 -1.08462080e+10]
 [-5.27212741e+12 -3.81662213e+12]
 [-1.66082852e+12 -6.81429172e+12]
 [ 1.13179833e+12 -3.82195886e+12]
 [ 8.68833296e+11 -2

In [31]:
lista1 = []
for item in pred2:
    print(max(item))

1849424000000.0
-2787425000000.0
-1372871700000.0
3557213600000.0
1993263300000.0
251487580000.0
-1873304900000.0
-1647920800000.0
-12847153000.0
2745883400000.0
-2530767500000.0
-8067866400000.0
6378992000000.0
-4212580700000.0
805872100000.0
-2685879200000.0
-3894968800000.0
6072097000000.0
-2304193900000.0
3009875300000.0
3961880300000.0
-633085100000.0
159072120000.0
-4270761400000.0
2714015600000.0
-3816622000000.0
-1660828500000.0
1131798300000.0
868833300000.0
-1707961900000.0
-3256185000000.0
826756560000.0
-1406102700000.0
-5346848400000.0
-9809229000000.0
-6774545400000.0
-4272063500000.0
1138780800000.0
2315928500000.0
3969217700000.0
-4027915700000.0
1986898400000.0
4369552200000.0
773072600000.0
-3864960400000.0
-311271360000.0
2219041600000.0
-4173445700000.0
-4172436700000.0
-799637770000.0
1397626900000.0
-500646800000.0
-4922586000000.0
-1828947200000.0
1552337100000.0
1335300200000.0
-929691600000.0
-3792068100000.0
3629368000000.0
-1525253400000.0
-2167496000000.0
15

In [32]:
lista = []
for item in pred2:
    lista.append(list(item).index(max(item)))
print(lista)

[1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]


In [33]:
l = []
for i in range(200):
    l.append(data1[i][1].index(1))
print(l)

[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]


In [34]:
y_test = lista
y_pred = l
matriz_conf = confusion_matrix(y_true = y_test,y_pred=y_pred)
print(matriz_conf)

[[75 25]
 [42 58]]
